In [439]:
import pandas as pd
import hashlib
import re

### Load

In [440]:
filename= "../data/export_15_03_2025_10_47_36.xls"
df = pd.read_excel(filename, skiprows=2)
df.head()

,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Pointage operation,Commentaire operation
0,14-03-2025,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,NaN,NaN
1,14-03-2025,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,NaN,NaN
2,14-03-2025,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,NaN,NaN
3,14-03-2025,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,NaN,NaN
4,13-03-2025,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,NaN,NaN


### Processing

Add extraction date timestamp in a column

In [441]:
_, day, month, year, hour, minute, second = filename.replace(".xls", "").split("_")
extracted_date = f"{year}-{month}-{day} {hour}:{minute}:{second}"
df["Date Extraction"] = extracted_date
df.head()

,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Pointage operation,Commentaire operation,Date Extraction
0,14-03-2025,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,NaN,NaN,2025-03-15 10:47:36
1,14-03-2025,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,NaN,NaN,2025-03-15 10:47:36
2,14-03-2025,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,NaN,NaN,2025-03-15 10:47:36
3,14-03-2025,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,NaN,NaN,2025-03-15 10:47:36
4,13-03-2025,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,NaN,NaN,2025-03-15 10:47:36


Convert types

In [442]:
df["Date operation"] = pd.to_datetime(df["Date operation"])
df['Date Extraction'] = pd.to_datetime(df['Date Extraction'])

/var/folders/64/lbsssmms29n9r2sh2cm9dxv00000gn/T/ipykernel_58485/2130239904.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Date operation"] = pd.to_datetime(df["Date operation"])


Add id

In [443]:
df['id'] = df['Date operation'].astype(str) + "_" + df['Categorie operation'].astype(str) + "_" + df['Sous Categorie operation'].astype(str) + "_" + df['Libelle operation'].astype(str) + "_" + df['Montant operation'].astype(str)
df['id'] = df['id'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
col = df.pop('id')
df.insert(0, 'id', col)

Add day/month/year

In [444]:
df["Jour"] = df["Date operation"].dt.day
df["Mois"] = df["Date operation"].dt.month
df["Année"] = df["Date operation"].dt.year
df["Mois-Annee"] = df["Date operation"].dt.strftime("%m-%Y")

Remove unused columns

In [445]:
df.drop(columns=['Pointage operation'], inplace=True)
df.drop(columns=['Commentaire operation'], inplace=True)

In [446]:
df['Libelle operation'].tolist()

['VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT TAMADON',
 'PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL - CARTE*5333',
 'PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/25 - CARTE*5333',
 'PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CARTE*5333',
 'PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CARTE*5333',
 'PAIEMENT CB HPY*LE COQ SPOR DU 08/03/25 A ROMILLY-SUR-S - CARTE*5333',
 'PAIEMENT CB NEXANS FRANCE S (FRANCE) DU 10/03/25 - CARTE*5333',
 'PAIEMENT CB OPENAI *CHATGPT (ETATS-UNIS) DU 06/03/25 - CARTE*5333',
 'PRELEVEMENT CHUBB EUROPEAN GROUP SE CHUBB EUROPEAN GROUP SE DU 10/03/25 - EMETTEUR : FR39ZZZ495463 MDT - MOTIF : 148699415 - REF : CIGSP70D 26',
 'PAIEMENT CB SUSHI HOUSE (FRANCE) DU 07/03/25 - CARTE*5333',
 'PAIEMENT CB BOULANGERIE MAR (FRANCE) DU 07/03/25 - CARTE*5333',
 'PAIEMENT CB POISSONNIERE (FRANCE) DU 06/03/25 - CARTE*5333',
 'PAIEMENT CB MEDIUM MONTHLY (ETATS-UNIS) DU 06/03/25 - CARTE*5333',
 'PAIEMENT CB APPLE (IRLANDE) DU 06/03/25 - CARTE*5333',
 'PAIEMENT CB CARAVANA (

Clean description

In [447]:
def clean_transaction_description(df):
    """
    Nettoie les descriptions des transactions en supprimant les dates (format DU JJ/MM/AA)
    et les références de carte (format - CARTE*XXXX).

    Args:
        df (pd.DataFrame): DataFrame contenant la colonne à nettoyer.
        column_name (str): Nom de la colonne à modifier.

    Returns:
        pd.DataFrame: DataFrame avec la colonne nettoyée.
    """
    df = df.copy()  # Pour éviter de modifier l'original
    df['Libelle clean'] = df['Libelle operation']

    # Expression régulière pour supprimer "DU JJ/MM/YY" et "- CARTE*XXXX"
    df['Libelle clean'] = df['Libelle clean'].str.replace(r"DU \d{2}/\d{2}/\d{2}", "", regex=True)
    df['Libelle clean'] = df['Libelle clean'].str.replace(r"- CARTE\*\d+", "", regex=True)

    # Nettoyage des espaces inutiles
    df['Libelle clean'] = df['Libelle clean'].str.replace(r"\s+", " ", regex=True).str.strip()

    return df

df = clean_transaction_description(df)
df.head()

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Date Extraction,Jour,Mois,Année,Mois-Annee,Libelle clean
0,77f8493ca24f02dea17cb67965c5258a73fc4fde75e7b2...,2025-03-14,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,2025-03-15 10:47:36,14,3,2025,03-2025,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...
1,942d2e6eae9a231cc72985bdc08804aada0b9d36d372a7...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,2025-03-15 10:47:36,14,3,2025,03-2025,"PAIEMENT CB ADOBE A SAGGART, DUBL"
2,941bb1735f80c2267aac1c36206a2eae90058e2f740d9a...,2025-03-14,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,2025-03-15 10:47:36,14,3,2025,03-2025,PAIEMENT CB BISTRO DU FAUBO (FRANCE)
3,a09fdabc5afb7f14f4275c530543500f1e1294e929fd2f...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,2025-03-15 10:47:36,14,3,2025,03-2025,PAIEMENT CB APPLE (IRLANDE)
4,4b45642f04eeb3b3f50599b08b28c403fe6cf684a80ae5...,2025-03-13,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,2025-03-15 10:47:36,13,3,2025,03-2025,PAIEMENT CB CARAVANA (FRANCE)


Detect single paiements in month

In [448]:
# def detect_single_month_payment_flag(df):
#     """
#     Ajoute une colonne 'is_single_month_payment' au DataFrame,
#     indiquant si une opération apparaît une seule fois dans le mois.

#     Args:
#         df (pd.DataFrame): DataFrame contenant une colonne 'Date operation' et 'Libelle operation'.

#     Returns:
#         pd.DataFrame: DataFrame avec la nouvelle colonne 'is_single_month_payment'.
#     """

#     # Compter les occurrences de chaque opération dans un mois donné
#     df["operation_count"] = df.groupby(["Mois-Annee", "Libelle clean"])["Libelle clean"].transform("count")

#     # Marquer les opérations uniques dans le mois
#     df["is_single_month_payment"] = df["operation_count"] == 1

#     # Supprimer la colonne temporaire
#     df.drop(columns=["operation_count"], inplace=True)

#     return df

# df = detect_single_month_payment_flag(df)
# df.head()

Detect subscription

In [449]:
import pandas as pd
from rapidfuzz import fuzz, process

def detect_subscriptions(df, min_months=3, date_col="Date operation", label_col="Libelle clean", amount_col="Montant operation"):
    """
    Détecte les abonnements dans un dataframe de transactions bancaires.

    - Normalise les libellés similaires avec fuzzy matching.
    - Regroupe les paiements récurrents par libellé et montant.
    - Identifie les transactions avec une périodicité proche de 30 jours.
    
    Args:
        df (pd.DataFrame): Le dataframe contenant les transactions.
        min_months (int): Nombre minimum de mois pour considérer un abonnement.
        date_col (str): Nom de la colonne contenant les dates.
        label_col (str): Nom de la colonne contenant les libellés des transactions.
        amount_col (str): Nom de la colonne contenant les montants des transactions.

    Returns:
        pd.DataFrame: Liste des abonnements détectés avec leur fréquence et leur périodicité moyenne.
    """
    
    # 1️⃣ Normalisation des libellés avec Fuzzy Matching
    unique_labels = df[label_col].unique()
    label_mapping = {}

    for label in unique_labels:
        best_match = process.extractOne(label, label_mapping.keys(), scorer=fuzz.token_sort_ratio)
        if best_match and best_match[1] > 85:  # Seulement si la similarité est élevée
            label_mapping[label] = label_mapping[best_match[0]]
        else:
            label_mapping[label] = label

    df["Libelle Normalisé"] = df[label_col].map(label_mapping)

    # 2️⃣ Extraction des dates et regroupement par mois
    df[date_col] = pd.to_datetime(df[date_col])
    df["Mois-Annee"] = df[date_col].dt.to_period("M")

    # 3️⃣ Détection des paiements récurrents
    grouped = df.groupby(["Libelle Normalisé", amount_col])["Mois-Annee"].nunique()
    abonnements = grouped[grouped >= min_months].reset_index()

    # 4️⃣ Calcul de la périodicité moyenne des paiements
    df = df.sort_values(["Libelle Normalisé", date_col])
    df["Diff Jour"] = df.groupby(["Libelle Normalisé", amount_col])[date_col].diff().dt.days

    # Calcul de l'écart moyen des paiements
    periodicity = df.groupby(["Libelle Normalisé", amount_col])["Diff Jour"].mean().reset_index()
    abonnements = abonnements.merge(periodicity, on=["Libelle Normalisé", amount_col], how="left")

    # Filtrage des abonnements avec des paiements espacés de 25 à 35 jours
    abonnements = abonnements[(abonnements["Diff Jour"] >= 25) & (abonnements["Diff Jour"] <= 35)]

    # Renommage pour clarté
    abonnements = abonnements.rename(columns={"Diff Jour": "Periodicite Moyenne (jours)", "Mois-Annee": "Nombre de Mois"})
    
    return abonnements


detect_subscriptions(df)
# df[df.is_subscription == True].head(20)

# .sort_values('Montant operation', ascending=False)

,Libelle Normalisé,Montant operation,Nombre de Mois,Periodicite Moyenne (jours)
0,COMMISSIONS COTISATION A UNE OFFRE GROUPEE DE ...,-3.14,11,30.500000
2,"PAIEMENT CB ADOBE A SAGGART, DUBL",-11.99,8,30.285714
3,PAIEMENT CB AMAZON PRIME FR A PAYLI2469664,-6.99,4,30.000000
4,PAIEMENT CB APPLE (IRLANDE),-11.99,5,30.750000
5,PAIEMENT CB APPLE (IRLANDE),-9.99,11,30.500000
7,PAIEMENT CB APPLE (IRLANDE),-2.99,13,29.750000
8,PAIEMENT CB CANAL PLUS A ISSY LES MOUL,-22.98,4,30.666667
10,PAIEMENT CB CLASSPASS* MONT (PAYS-BAS),-5.00,7,30.666667
13,PAIEMENT CB NAVIGO (FRANCE),-88.80,3,28.000000
14,PAIEMENT CB NAVIGO (FRANCE),-86.40,4,28.333333


In [464]:
count_mois = df.groupby(['Libelle clean', 'Montant operation'])['Mois-Annee'].nunique().reset_index()
count_mois.sort_values('Mois-Annee', ascending=False).head(50)

,Libelle clean,Montant operation,Mois-Annee
57,PAIEMENT CB APPLE (IRLANDE),-3.99,13
58,PAIEMENT CB APPLE (IRLANDE),-2.99,13
499,PAIEMENT CB WTA TV (ROYAUME-UNI),-8.93,11
56,PAIEMENT CB APPLE (IRLANDE),-9.99,11
1,COMMISSIONS COTISATION A UNE OFFRE GROUPEE DE ...,-3.14,11
589,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,10
15,"PAIEMENT CB ADOBE A SAGGART, DUBL",-11.99,8
138,PAIEMENT CB CLASSPASS* MONT (PAYS-BAS),-5.00,7
329,PAIEMENT CB NAVIGO A PARIS 9,-86.40,6
2,COMMISSIONS SERVICE AFFINITE,-12.00,6


In [453]:
pd.DataFrame(df['Libelle clean'].value_counts()).head(50)

,count
Libelle clean,
PAIEMENT CB APPLE (IRLANDE),52
PAIEMENT CB LYOVEL SAS (FRANCE),14
COMMISSIONS COTISATION A UNE OFFRE GROUPEE DE SERVICES ESPRIT LIBRE,13
PAIEMENT CB AWS EMEA (LUXEMBOURG),13
PAIEMENT CB AMAZON A PAYLI2441535,12
PAIEMENT CB BALTAIAN ET COM (FRANCE),12
PAIEMENT CB WTA TV (ROYAUME-UNI),11
PAIEMENT CB FRANPRIX (FRANCE),10
VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE MOULA AUGMENTE,10


In [451]:
df[(df['Libelle operation'].str.contains(r'cantine', case=False, regex=True))]

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Date Extraction,Jour,Mois,Année,Mois-Annee,Libelle clean,Libelle Normalisé
35,cb2cba6321e83d7f4e5313dc526a3ead399f45317e9637...,2025-02-24,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB MA CANTINE CORE (FRANCE) DU 20/02/...,-11.0,2025-03-15 10:47:36,24,2,2025,2025-02,PAIEMENT CB MA CANTINE CORE (FRANCE),PAIEMENT CB MA CANTINE CORE (FRANCE)
37,29213fa1cc72481d94362b5231d513af16eb0b16a1bae7...,2025-02-21,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB MA CANTINE CORE (FRANCE) DU 19/02/...,-11.0,2025-03-15 10:47:36,21,2,2025,2025-02,PAIEMENT CB MA CANTINE CORE (FRANCE),PAIEMENT CB MA CANTINE CORE (FRANCE)


In [452]:
df[(df.is_subscription == True) & (df['Libelle operation'].str.contains(r'cantine', case=False, regex=True))]

AttributeError: 'DataFrame' object has no attribute 'is_subscription'

In [ ]:
test = df[(df.is_subscription == True) & (df['Libelle operation'].str.contains(r'cantine', case=False, regex=True))]
monthly_counts = test.groupby(['Montant operation', 'Mois'])['Libelle operation'].count().reset_index()
valid_monthly = monthly_counts[monthly_counts['Libelle operation'] == 1]
valid_monthly

,Montant operation,Mois,Libelle operation


In [ ]:
test['Libelle operation'].tolist()

['PAIEMENT CB MA CANTINE CORE (FRANCE) DU 20/02/25 - CARTE*5333',
 'PAIEMENT CB MA CANTINE CORE (FRANCE) DU 19/02/25 - CARTE*5333']

### Export

In [ ]:
df.head()

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Date Extraction,Jour,Mois,Année,Mois-Annee,Libelle clean,is_single_value,is_subscription
0,77f8493ca24f02dea17cb67965c5258a73fc4fde75e7b2...,2025-03-14,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,2025-03-15 10:47:36,14,3,2025,03-2025,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,True,False
1,942d2e6eae9a231cc72985bdc08804aada0b9d36d372a7...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,2025-03-15 10:47:36,14,3,2025,03-2025,"PAIEMENT CB ADOBE A SAGGART, DUBL",True,True
2,941bb1735f80c2267aac1c36206a2eae90058e2f740d9a...,2025-03-14,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,2025-03-15 10:47:36,14,3,2025,03-2025,PAIEMENT CB BISTRO DU FAUBO (FRANCE),True,False
3,a09fdabc5afb7f14f4275c530543500f1e1294e929fd2f...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,2025-03-15 10:47:36,14,3,2025,03-2025,PAIEMENT CB APPLE (IRLANDE),False,True
4,4b45642f04eeb3b3f50599b08b28c403fe6cf684a80ae5...,2025-03-13,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,2025-03-15 10:47:36,13,3,2025,03-2025,PAIEMENT CB CARAVANA (FRANCE),False,False


In [ ]:
df.dtypes

id                                  object
Date operation              datetime64[ns]
Categorie operation                 object
Sous Categorie operation            object
Libelle operation                   object
Montant operation                  float64
Date Extraction             datetime64[ns]
Jour                                 int32
Mois                                 int32
Année                                int32
Mois-Annee                          object
Libelle clean                       object
is_single_value                       bool
is_subscription                       bool
dtype: object

In [ ]:
df.to_excel("../data/bnp_export_15_03.xlsx", index=False)

### Bin

In [ ]:
def detect_subscriptions_v0(df):
    """
    Ajoute une colonne 'is_subscription' au dataframe en détectant les paiements récurrents 
    sur au moins 3 mois consécutifs avec exactement le même montant.

    :param df: DataFrame contenant au moins les colonnes 'Date operation' et 'Montant operation'
    :return: DataFrame avec la colonne 'is_subscription' ajoutée
    """

    # Comptage du nombre d'occurrences par mois et par montant
    monthly_counts = df.groupby(['Montant operation', 'Mois-Annee'])['Libelle clean'].count().reset_index()

    # Filtrage des montants qui apparaissent exactement 1 fois par mois
    valid_monthly = monthly_counts[monthly_counts['Libelle clean'] == 1]

    # Comptage du nombre de mois distincts pour chaque montant
    subscription_candidates = valid_monthly.groupby('Montant operation')['Mois-Annee'].nunique().reset_index()

    # Identifier les montants correspondant à un abonnement (au moins 3 mois consécutifs)
    subscription_candidates = subscription_candidates[subscription_candidates['Mois-Annee'] >= 3]

    # Ajouter la colonne is_subscription au dataframe original
    df['is_subscription'] = df['Montant operation'].isin(subscription_candidates['Montant operation'])

    return df